In [11]:
from PublicDataReader import Fred
import PublicDataReader as pdr
api_key = 'J3mom58lVV/iYvvqpYGAr6SKocXDz3n52lMWM05YbPu10ZawRGcGm3e2f6VU7AESR+TJFOjeI1abx4ViWRE1NQ=='
api = pdr.Fred(api_key=api_key)
# api.code_bdong()
bdong = pdr.code_bdong()
bdong = bdong[bdong['시도명'] == '강원특별자치도']
print(bdong)



출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.8.1. 시행)
      시도코드      시도명  시군구코드 시군구명       법정동코드 읍면동명  동리명      생성일자 말소일자
46498   51  강원특별자치도  51000       5100000000            20230611     
46500   51  강원특별자치도  51110  춘천시  5111000000            20230611     
46501   51  강원특별자치도  51110  춘천시  5111010100  봉의동       20230611     
46502   51  강원특별자치도  51110  춘천시  5111010200  요선동       20230611     
46503   51  강원특별자치도  51110  춘천시  5111010300  낙원동       20230611     
...    ...      ...    ...  ...         ...  ...  ...       ...  ...
47967   51  강원특별자치도  51830  양양군  5183035038  강현면  장산리  20230611     
47968   51  강원특별자치도  51830  양양군  5183035039  강현면  전진리  20230611     
47969   51  강원특별자치도  51830  양양군  5183035040  강현면  주청리  20230611     
47970   51  강원특별자치도  51830  양양군  5183035041  강현면  적은리  20230611     
47971   51  강원특별자치도  51830  양양군  5183035042  강현면  물갑리  20230611     

[1473 rows x 9 columns]


In [3]:
import json, os, sys
import requests
import xml
import PublicDataReader as TransactionPrice
import numpy as np


API_KEY = 'J3mom58lVV/iYvvqpYGAr6SKocXDz3n52lMWM05YbPu10ZawRGcGm3e2f6VU7AESR+TJFOjeI1abx4ViWRE1NQ=='
columns_dict = {
        'sggCd': '법정동시군구코드', 
        'umdCd': '법정동읍면동코드', 
        'landCd': '법정동지번코드', 
        'bonbun': '법정동본번코드', 
        'bubun': '법정동부번코드', 
        'roadNm': '도로명', 
        'roadNmSggCd': '도로명시군구코드', 
        'roadNmCd': '도로명코드', 
        'roadNmSeq': '도로명일련번호코드', 
        'roadNmbCd': '도로명지상지하코드', 
        'roadNmBonbun': '도로명건물본번호코드', 
        'roadNmBubun': '도로명건물부번호코드', 
        'umdNm': '법정동', 
        'aptNm': '단지명', 
        'jibun': '지번', 
        'excluUseAr': '전용면적', 
        'dealYear': '계약년도', 
        'dealMonth': '계약월', 
        'dealDay': '계약일', 
        'dealAmount': '거래금액', 
        'floor': '층', 
        'buildYear': '건축년도', 
        'aptSeq': '단지일련번호', 
        'cdealType': '해제여부', 
        'cdealDay': '해제사유발생일', 
        'dealingGbn': '거래유형', 
        'estateAgentSggNm': '중개사소재지', 
        'rgstDate': '등기일자', 
        'aptDong': '아파트동명', 
        'slerGbn': '매도자', 
        'buyerGbn': '매수자', 
        'landLeaseholdGbn': '토지임대부아파트여부', 
    }
api = TransactionPrice.TransactionPrice(service_key=API_KEY)
df = api.get_data(
        property_type='아파트',
        trade_type='매매',
        sigungu_code='51230',   # 51180, 51820
        start_year_month='201911',
        end_year_month='202411'
    )
print(df)
# if df.empty:
#     print('df is empty')
# else:
#     try:
#         # 컬럼명 매핑 시도
#         df.columns = [columns_dict.get(c, c) for c in df.columns]
#     except Exception as e:
#         print(f"컬럼 매핑 중 오류 발생: {e}")
#         print(f"원본 컬럼: {df.columns}")
#     df = df.replace({np.nan: None})   
#     print(df)



      sggCd  umdCd landCd bonbun bubun roadNm roadNmSggCd roadNmCd roadNmSeq  \
0     51230  10600      1   0501  0005   동해대로       51230  2000023        01   
1     51230  10600      1   0757  0000    정상로       51230  3224027        00   
2     51230  11700      1   0176  0000   엑스포로       51230  3224037        03   
3     51230  10600      1   0733  0001    교동로       51230  3223002        00   
4     51230  10600      1   0745  0001    정상로       51230  3224027        01   
...     ...    ...    ...    ...   ...    ...         ...      ...       ...   
3239  51230  10600      1   0757  0000    정상로       51230  3224027        00   
3240  51230  11700      1   0176  0000   엑스포로       51230  3224037        03   
3241  51230  10600      1   0793  0000    하실길       51230  4472344        00   
3242  51230  12000      1   0055  0117    중앙로       51230  3224040        03   
3243  51230  11800      1   0117  0000   오십천로       51230  3224022        06   

     roadNmbCd  ... cdealType cdealDay 

In [2]:
import redis
import json
import time

redis_client = redis.Redis(
    host='localhost',
    port=6379,
    db=0,
    decode_responses=True  # UTF-8 디코딩
)

pubsub = redis_client.pubsub()
channel = "stock:nws"
pubsub.subscribe(channel)
print(f"Subscribed to news channel: {channel}")

# # 구독 메시지 처리
# first_message = pubsub.get_message()
# print("구독 메시지:", first_message)

while True:
    message = pubsub.get_message()
    if message and message['type'] == 'message':
        # 메시지 데이터를 JSON으로 파싱
        news_data = json.loads(message['data'])
        print("수신된 뉴스:", news_data['body']['title'])
        print("전체 데이터:", news_data)
    time.sleep(0.1)  # CPU 사용량 감소

Subscribed to news channel: stock:nws
수신된 뉴스: 나이키(NKE), 실적 리스크 여전하지만 장기적 회복 예상할 수 있어 - UBS
전체 데이터: {'header': {'tr_cd': 'NWS', 'tr_key': 'NWS001'}, 'body': {'date': '20241226', 'code': '', 'realkey': '202412262107163410002053', 'bodysize': '513', 'time': '210716', 'id': '34', 'title': '나이키(NKE), 실적 리스크 여전하지만 장기적 회복 예상할 수 있어 - UBS'}}
수신된 뉴스: '尹탄핵심판 첫날 방청 경쟁률 '2251대 1'....2만여명 몰려
전체 데이터: {'header': {'tr_cd': 'NWS', 'tr_key': 'NWS001'}, 'body': {'date': '20241226', 'code': '', 'realkey': '202412262109333005385351', 'bodysize': '3787', 'time': '210933', 'id': '30', 'title': "'尹탄핵심판 첫날 방청 경쟁률 '2251대 1'....2만여명 몰려"}}
수신된 뉴스: [인사] KB자산운용
전체 데이터: {'header': {'tr_cd': 'NWS', 'tr_key': 'NWS001'}, 'body': {'date': '20241226', 'code': '', 'realkey': '202412262109472600000990', 'bodysize': '3736', 'time': '210947', 'id': '26', 'title': '[인사] KB자산운용'}}
수신된 뉴스: 세계은행, 올해 中성장률 전망 4.8%→4.9% 상향
전체 데이터: {'header': {'tr_cd': 'NWS', 'tr_key': 'NWS001'}, 'body': {'date': '20241226', 'code': '', 'realkey': '2

In [ ]:
'''
 <img src="https://biz.chosun.com/resizer/v2/QP4M6YLT45GNLE3TSOY32KW72Y.jpg?auth=9f9959a357adccf20f43a932d01ff65db16962cd644ef2c290612be40dae9545&width=2980&height=1680&smart=true" width="2980">
“오행을 알면 나를 이해한다. 내가 만들어가는 오늘의 운세”목(木), 화(火), 토(
土), 금(金), 수(水)로 이루어진 오행은 동양 철학에서 인간과 자연을 설명하는 중
요한 원리입니다. 사람의 성향을 �保� 다섯 가지로만 구분할 수는 없지만, 오행은 
만물의 근본을 이루는 에너지를 상징하며, 이를 통해 개�括� 기질과 운세를 분석할 
수 있습니다. 오행을 깊이 이해한다면, 자신을 포함한 세상에 대한 통찰이 더�� 명
확해질 것입니다. 오늘의 운세는 단순한 예측이 아닌, 내가 만들어가는 하루의 지침
입니다. 이를 �淪� 더 나은 결정을 내릴 수 있는 출발점이 되기를 바랍니다.
<img src="https://biz.chosun.com/resizer/v2/5GQWX5VV4REODCDJROIFSH7P64.jpg?auth=4427a4665e52fe31f161f4e944498c5dc9be0e01e546968a23b03cdbb19d557b&width=2980&height=1680&smart=true" width="2980">
오늘은 갑진년(甲午年) 병자월(丙子月) 을축일(乙丑日)로, 기운은 수(水)와 토(土)
가 중심을 이루며 목(木)과 금(金)의 균형이 미세하게 조정되는 흐름입니��. 갑목(
甲木)의 뿌리와 축토(丑土)의 기운이 얽히며 오행 간의 상생과 상극이 복합적으로 
작용합니다.아래는 을축일에 각 오행별 오늘의 운세, 특징, 주의할 점과 개선 방법
입니다.● 목(木) 오행 오늘의 운세- 목(木) 오행 일운 주의사항오늘의 일간인 을목
(乙木)은 축토(丑土)의 습토(濕土) 기운과 마주하며 성장이 일부 제약받는 흐름을 
보입니다. 축토는 을목을 잠시 묶어두지만, 상극의 극단적 충돌보다는 균형을요구
하는 기운입니다. 한편, 병자월의 자수(子水)가 목(木)을 생조(生助)하며 근본적인 
기운은 안정적입니다. 오늘은 확장과 고집을 경계하고 주변과의 원만한 관계를 유지
하는 것이 중요합니다. 학문, 연구,창작과 같은 자기계발에 유리한 흐름이니 서두
르지 말고 차근차근 목표를 향해 나아가는 것이 좋습니다.다만, 목(木)의 과잉은 
토(土)와 충돌을 일으킬 수 있으므로 균형 감각이 필요합니다.- 목(木) 오행, “지
나친 자기 주장은 삼가세요.”오늘의 핵심은 유연함과 조화입니다. 자신의 생각이나
 주장을 강하게 �棘咀牡堅羞릿募� 주변과의 협력과 소통을 우선시해야 합니다. 타협
과 양보를 통해 더 큰 성과를 이끌어낼수 있는 날입니다. 건강 관리도 신경 써야 
합니다. 간장은 목(木) 기운과 밀접한 관련이 있으므로 스트�뭣보� 줄이고 과로를 
피하세요. 충분한 휴식과 심신의 안정을 유지하는 것이 중요합니다.수(水)의 기운을
 강화하여 목(木)의 성장을 뒷받침하는 것이 필요합니다. 반신욕을 통해 기운을 보
충하세요.● 화(火)오행 오늘의 운세- 화(火) 오행 일운 주의사항오늘은 병자월(丙
子月)에 병화(丙火)가 월간에 떠 있지만,자수(子水)의 강력한 기운이 화(火)를 제
압하고 기세를 약화시키는 날입니다. 을축일(乙丑日)의 축토(丑��)가 중화의 역할을
 하지만 화(火)를 온전히 보완하지는 못하므로 전반적으로 기운이 약하고 저조한 흐
름을 보입니다. 화(火)의 기운이 약해지면 감정적 무기력이나 의욕 저하로 이어질 
수 있으므로 오늘은 불필요한 감정 소모를 피하고 내면을 차분히 가다듬는 것이 중
요합니다. 의욕을 앞세우기보다는 자기계발이나 내면 수양에 집중하는 하루를 보내
세요.- 화(火) 오행, “생강차 한 잔 하실래요?”오늘의 핵심은 체�� 유지와 내적 
안정입니다. 화(火)의 기운이 약한 날이므로 타인과의 불필요한 논쟁이나 갈등은 피
해야합니다. 감정적으로 격앙되면 오히려 손해를 보게 될 수 있으니 평정심을 유지
하고 차분히 상황을 바라�릿� 자세가 필요합니다.건강 관리에도 각별히 유의해야 합
니다. 화(火)는 심장(心臟)과 혈액 순환을 주관하므로 체온이 떨어지지 않도록 주의
하고 따뜻한 음식을 섭취해 몸을 보호하세요. 호박죽, 된장국, 생강�殆� 같이 따뜻
하고 소화에 좋은 음식을 섭취하세요.● 토(土) 오행 오늘의 운세- 토(土) 오행 일
운 주�퓨聆六윱쳄� 을축일(乙丑日)의 축토(丑土)가 중심에 서 있으며, 축토는 습하
고 무겁게 자리하면서 병자월(丙子月)의 수(水) 기운과 맞물려 약간의 불안정성을 
보입니다. 그러나 갑진년(甲辰年)의 진토(辰土)가 ��(土)의 중심을 잡아주어 흐름이
 무너지지 않고 균형을 유지하는 날입니다. 토(土)의 특성인 중재와 균형의 기운이 
발현되기에 실무적이거나 실질적인 문제를 해결하기 좋은 날입니다. 다만 수(水)의 
기운이 �駭謀球퓐� 불필요한 금전적 욕심이나 무리한 결정은 피해야 합니다. 현실적
이고 신중한 태도로 접근하면손실을 막고 안정감을 유지할 수 있습니다.- 토(土) 
오행, “무리한 투자를 주의하세요”오늘은 금전 관��와 절제가 핵심입니다. 섣부른
 판단으로 큰 투자를 하거나 무리한 재정 운용을 시도하면 작은 실수가 큰손해로 
이어질 수 있습니다. 신중하게 상황을 판단하고 안전한 길을 선택하는 것이 좋습니
다. 또한, ��(土)는 비장과 위장을 주관하므로 소화기 건강에 유의해야 합니다. 과
식이나 자극적인 음식은 피하고 소화에 부담이 가지 않도록 가볍고 따뜻한 식사를 
하세요. 오늘은 금(金)의 기운을 활용하여 불안정한 토(��)를 보완하는 것이 필요합
니다. 금속 장신구를 몸에 지니면 토(土)의 흐름을 다스리는 데 유리합니다.●금(
金) 오행 오늘의 운세- 금(金) 오행 일운 주의사항오늘의 금(金)은 병자월(丙子月)
의 자수(子水)가금(金)을 생조(生助)하면서 기운을 강하게 유지하지만, 을목(乙木)
의 기운이 금(金)을 극(剋)하면서 긴�弱㉯� 흐르는 날입니다. 이는 책임감과 완벽주
의로 이어질 수 있어 심리적으로 부담을 느끼기 쉽습니다.�윱쳄� 자신의 강한 에너
지와 능률을 체계적으로 활용하면 생산적이고 실속 있는 성과를 얻을 수 있습니다. 
다만 과도한 완벽주의나 압박감에 사로잡히면 오히려 역효과를 불러올 수 있으니 균
형과 조절을 잃지않는 것이 중요합니다.- 금(金) 오행, “적절한 휴식으로, 따뜻하
게 체온을 유지하세요”오늘은 자기 비판과 스트레스를 경계해야 합니다. 완벽을 추
구하려는 성향이 지나치면 몸과 마음의 균형이 무너질 수 있으니 스스로를 다독이고
 여유를 가지세요. 건강 관리가 특히 중요합니다. 금(金)은 폐(肺)와 호흡기와 연결
되어 있어 차가운 기운에 약하므로, 따뜻하게 체온을 유지하고 기관지 건강에 신경 
써야 합니다. 특��, 냉기로 인한 불편함이 있을 수 있으니 따뜻한 차나 음식으로 몸
을 보호하세요.● 수(水) 오행 오늘의운세- 수(水) 오행 일운 주의사항오늘은 병자
월(丙子月)과 을축일(乙丑日)의 자수(子水)가 강하게 작용하면서 수(水)의 기운이 
왕성하게 흐르는 날입니다. 그러나 축토(丑土)의 습한 기운이 수(水)의 흐름을 일��
 막아 약간의 불안정성을 더합니다. 이로 인해 지혜와 통찰력은 극대화되지만, 지나
친 고민과 우유부�洑纛� 오히려 운세를 흐트러뜨릴 수 있습니다.- 수(水) 오행, “
우선순위를 정리하세요”오늘은 감정 기�� 조절이 필수적입니다. 수(水)의 기운이 
강하면 내면의 불안이나 과도한 고민이 생길 수 있으니, 현재의상황에 집중하고 지
나친 걱정은 내려놓으세요. 오늘은 목(木)의 기운을 활용하여 수(水)의 흐름을 원활
하게 다스리는 것이 필요합니다.자연 속 산책이나 녹색 계열의 의상을 활용하여 목(
木)의 생기와 기운을받아들이세요. 또한 명확한 목표를 설정하고 실천 가능한 단계
를 구체화하세요. 우선순위를 정리하면 우유부단함이 해소되고 계획이 순조롭게 진
행됩니다.※ 운세서비스의 저작권은 사주나루에 있습니다.무단전재 및 재배포 금지

조선비즈 enternews@chosunbiz.com

Copyrights ⓒ ChosunBiz.com

'''

In [27]:
import websockets
import asyncio
import json
import nest_asyncio
nest_asyncio.apply()
 
async def connect_websocket():
    uri = "ws://localhost:8002/stock/ws_redis/news"
    uri = "wss://api2410.ebesesk.synology.me/stock/ws_redis/news"
    
    while True:  # 연결이 끊어지면 재연결 시도
        try:
            async with websockets.connect(uri) as websocket:
                print("WebSocket에 연결되었습니다.")
                
                # 연결 유지를 위한 무한 루프
                while True:
                    try:
                        message = await websocket.recv()
                        data = json.loads(message)
                        print(f"수신된 뉴스 제목: {data['body']['title']}")
                        print(f"전체 데이터: {data}")
                    except websockets.exceptions.ConnectionClosed:
                        print("연결이 종료되었습니다. 재연결 시도...")
                        await asyncio.sleep(1)  # 재연결 전 1초 대기
                        break
                    except Exception as e:
                        print(f"오류 발생: {e}")
                        await asyncio.sleep(1)
                        continue
                    
                    await asyncio.sleep(0.1)  # CPU 사용량 조절
                    
        except Exception as e:
            print(f"연결 오류: {e}")
            await asyncio.sleep(1)  # 재연결 전 1초 대기
            continue

# 비동기 이벤트 루프 실행
asyncio.get_event_loop().run_until_complete(connect_websocket())

연결 오류: 


KeyboardInterrupt: 

연결 오류: 
연결 오류: 
연결 오류: 
연결 오류: 
연결 오류: 
연결 오류: 
연결 오류: 


In [ ]:
["법정동시군구코드","법정동읍면동코드","법정동지번코드","법정동본번코드","법정동부번코드","도로명","도로명시군구코드","도로명코드","도로명일련번호코드","도로명지상지하코드","도로명건물본번호코드","도로명건물부번호코드","법정동","단지명","지번","단지명","지번","전용면적","계약년도","계약월","계약일","거래금액","층","건축년도","단지일련번호","해제여부","해제사유발생일","거래유형","중개사소재지","등기일자","아파트동명","매도자","매수자","토지임대부아파트여부",]

In [ ]:
    img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']
manga_extensions = ['.cbz', '.cbr', '.cb7', '.cbt', '.cbg', '.cbb', '.cbf']

src_ext = img_extensions + zip_extensions + manga_extensions
print(src_ext)

In [ ]:
'''
메인 디렉토리가 /home/manga

/home/manga/(장르 별 하위 디렉토리)/(manga 디렉토리)/(이미지 파일들(압축파일도 있을 수 있음))
예시 
/home/manga/manga/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__manga2/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__그림그리기/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
등등 여러개 있음

...

manga 확장자는 
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']

manga 압축파일 
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']

1. 압축파일있는 디렉토리와 image 파일있는 디렉토리는 제외하고 이미지 파일이 없거나 이미지 파일 수가 10개 미만인 디렉토리를 찾아서 하위 디렉토리와 파일들은 /home/manga/__del 디렉토리로 이동하고 삭제
2. 압축파일이있는 디렉토리중 그림파일이 2개 이하면 삭제하고 압축파일에서 한장씩만 문자 안깨지게 UTF-8 인코딩으로 같은 디렉토리에 복사
3. manga 디렉토리 아래 이미지 파일들 숫자.jpg 숫자.webp 형식으로 이름 변경
4. sqlite3 데이터베이스 만들고 디렉토리 정보 저장
    'folder' : manga 디렉토리의 /home/manga/ 부분 제외한 디렉토리 경로
    'images' : manga 디렉토리 아래 이미지 파일 목록
    'zip_files' : manga 디렉토리 아래 압축파일 목록
    'file_date' : 이미지 파일들 생성일자 중 가장 오래된 날짜
    'create_date' : db 생성일자
    'update_date' : db 수정일자
    'tag' : ''
    
    
    
'''


In [ ]:
import re, os
filename = 'aaa/몰라_01-001.jpg'
basename = os.path.basename(filename)
print('basename', basename)
name, ext = os.path.splitext(basename)
print('name', name)
print('ext', ext)
number_part = re.findall(r'\d+$', name)[0]  # 뒷부분 숫자 선택
prefix_part = re.findall(r'_?([\d]{2})-', name)[0]  # 앞부분 숫자 선택
title_part = name.replace(prefix_part, '').replace(number_part, '')    # 뒷부분 숫자 제거
print('prefix_part', prefix_part)
print('number_part', number_part)
print('title_part', title_part)


In [ ]:
import re, os

def get_volume(text: str) -> str:
    patterns = [
        r'_[^_]+편_\d{1,3}$',
        r'(_\d{1,3}_\d{1,3}_\d{1,3}_\d{1,3}$)',
        r'(_\d{1,3}_\d{1,3}_\d{1,3}$)',
        r'(_\d{1,3}_\d{1,3}$)',
        r'(_[\d]{1,3})$',
        r'_\d{1,3}_무삭제판?$',
        r'_\d{1,3}_[상하]$',
        # r'_(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3})$',
        # r'_(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3})$',
        r'(_[上下상하])_',
        r'(_[上下상하])$',
        # r'(_[상하])_',
        # r'(_[상하])$',
        r'(_[^_]{1,5}[편권판])$',
        r'_[vV][oO][lL][uU]?[mM]?[eE]?(_?\d{1,3})_?',
        r'_[vV][oO][lL][uU]?[mM]?[eE]?(_?\d{1,3})$  ',
        r'_[eE][pP][iI][sS][oO][dD][eE](_?\d{1,3})_?',
        r'_[eE][pP][iI][sS][oO][dD][eE](_?\d{1,3})$',
        r'_[eE][pP](_?\d{1,3})_?',
        r'_[eE][pP](_?\d{1,3})$',
        r'(_\d{1,3})_',
        r'(_\d{1,3})$',
        r'_[sS][eE][rR][iI][eE][sS](_?\d{1,3})_?',
        r'_[sS][eE][rR][iI][eE][sS](_?\d{1,3})$',
        r'_[pP][aA][rR][tT](_?\d{1,3})_?',
        r'_[pP][aA][rR][tT](_?\d{1,3})$',
        r'_[cC][hH]{0,2}(_?\d{1,3}_?\d{1,3})_?',
        r'(_제?\d{1,3}화)_?',
        r'(_\d{1,3}회)_?',
        r'_[sS][eE][rR][iI][eE][sS]_[cC](_\d{1,3})$',
        r'_\d{1,3}_?무삭제',
        r'_1st$',
        r'_2nd$',
        r'_3rd$',
    ]
    for p in patterns:
        result = re.findall(p, text)
        if result:
            return result
    return ''
        

def get_eng_title(text: str) -> str:

    patterns = [
        r'[가-힣]+_\d+$',
        r'[가-힣]_?\d+_?[가-힣]',
        r'[가-힣]+_',
        r'[^a-zA-Z0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_kor_title(text: str) -> str:
    patterns = [
        r'[^가-힣0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_jpn_title(text: str) -> str:
    patterns = [
        r'[^0-9ぁ-んァ-ヶ\u3400-\u4dbf\u4e00-\u9fff0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_kor_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9가-힣]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_jpn_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_kor_jpn_title(text: str) -> str:
    patterns = [
        r'[^가-힣0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_kor_jpn_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff가-힣]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_only_kor_first_title(text: str) -> str:
        patterns = [
            r'[^가-힣]',
            r'[\s]+'
        ]
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        
        return text.strip().split(' ')[0]

def get_title(text: str) -> str:
    texts = {
        'original': text,
        'eng': get_eng_title(text), 
        'kor': get_kor_title(text), 
        'jpn': get_jpn_title(text), 
        'eng_kor': get_eng_kor_title(text), 
        'eng_jpn': get_eng_jpn_title(text), 
        'kor_jpn': get_kor_jpn_title(text), 
        'eng_kor_jpn': get_eng_kor_jpn_title(text),
    }
    # texts['only_eng'] = texts['eng_kor'].split('_'+get_only_kor_first_title(text))[0]
    # # print("texts['only_eng']:", texts['only_eng'])
    words_cnt = [texts[t] for t in texts if len(texts[t].split('_')) > 1]
    words_cnt.sort(key=lambda x: len(x.split('_')))
    # print(words_cnt)
    _length = [len(texts[t]) for t in texts if len(texts[t]) > 0]
    max_length = max(_length)
    min_length = min(_length)
    max_text = [texts[t] for t in texts if len(texts[t]) == max_length][0]
    min_text = [texts[t] for t in texts if len(texts[t]) == min_length][0]
    
    if len(texts['eng_kor_jpn']) < 31:
        return texts['eng_kor_jpn']
    elif len(texts['eng'].split('_')) > 3:
        return texts['eng']
    elif texts['kor']:
        return texts['kor']
    elif texts['jpn']:
        return texts['jpn']
    else:
        return min_text
    
        
base_path = '/home/manga/manga3'
# for sub_path in os.listdir(base_path):
#     # print(sub_path)
#     if get_volume(sub_path):
#         print(get_volume(sub_path)[1].ljust(10, ' '), get_volume(sub_path)[0].ljust(30, ' '), sub_path)

for sub_path in os.listdir(base_path):
    if os.path.isdir(os.path.join(base_path, sub_path)):# and len(get_title(sub_path)) >3:
        # print(sub_path)
        try:
            patterns = [
                r'_?korean_팀_[가-힣]+$',
                r'_korean$',
            ]  
            for pattern in patterns:
                sub_path = re.sub(pattern, '', sub_path)    
            if get_volume(sub_path):    
                print(get_title(sub_path),'|', f"==>{get_volume(sub_path)[0]}<==", sub_path)
                print("==============================")
            else:
                print(get_title(sub_path), '|', sub_path)
        except Exception as e:
            print(e, sub_path, '+++++++++++++++++++++++++++++++++++++++++++')
        


In [ ]:
{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9ac9a75670>, 
 'height': 720, 
 'face': None, 
 'pregnant': None, 
 'uniform': None, 
 'showtime': 577, 
 'look': None, 
 'conversation': None, 
 'family': None, 
 'bitrate': 345561, 
 'age': None, 
 'lesbian': None, 
 'ad_start': None, 
 'id': 7926, 
 'filesize': 24922302, 
 'pussy': None, 
 'ani': None, 
 'ad_finish': None, 
 'cdate': datetime.datetime(2022, 6, 25, 10, 40, 49), 
 'etc': None, 'oral': None, 'star': None, 'display_quality': None, 
 'school_uniform': None, 'masturbation': None, 'date_posted': datetime.date(2022, 6, 27), 
 'dbid': 'test2/나스 왜 써요- 시놀로지 나스(NAS) 개념부터 구글 드라이브 비교 장단점, 나스 선택법까지 한 번에! - 나도 샀다 NAS EP01.mp4', 
 'country': None, 
 'hip': None, 
 'massage': None, 
 'date_modified': datetime.date(2022, 7, 21), 
 'width': 1280, 
 'group': None}

In [ ]:
import re, json

text = "file_1편 file_2편 file_3편"

# 1. search(): 첫 번째 매치만 찾음
search_result = re.search(r'_(\d+)편', text)
if search_result:
    print("search 결과:", search_result.group(1))  # "1"
    print("위치:", search_result.span())           # 매치된 위치
    print("전체 매치:", search_result.group(0))    # 전체 매치 문자열
    
search_result = re.findall(r'_(\d+)편', text)
print(search_result)


t = 'abc_34a_fd'
if 'c_3' in t:
    print('ok')
    print(t[:50])
a = []
print(a[:-1])


etc= '{"etc":"sod,sdjs,sdmu"}'
etc = json.loads(etc)
print(type(etc))
sub = '#쿼리#은아'
c = r'^#[a-zA-Z0-9가-힣]{2,5}+#'
_subject = re.sub(c, '', sub)
print(_subject)

In [ ]:
import os
import shutil
import sqlite3
from datetime import datetime
import pytz
import zipfile
import py7zr
import rarfile
import patoolib
from pathlib import Path

# 한국 시간대 설정
KST = pytz.timezone('Asia/Seoul')

# 파일 확장자 정의
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz', '.alz']

def get_file_list(directory):
    """디렉토리 내의 이미지와 압축 파일 목록을 반환"""
    images = []
    zip_files = []
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            ext = os.path.splitext(file)[1].lower()
            if ext in img_extensions:
                images.append(file_path)
            elif ext in zip_extensions:
                zip_files.append(file_path)
                
    return images, zip_files

def process_manga_directory(manga_path):
    """개별 만화 디렉토리 처리"""
    images, zip_files = get_file_list(manga_path)
    
    # 조건 1: 이미지가 없거나 10개 미만이고 압축파일이 없는 경우
    if len(zip_files) == 0 and (len(images) == 0 or len(images) < 10):
        del_path = os.path.join('/home/manga/__del', os.path.relpath(manga_path, '/home/manga'))
        os.makedirs(os.path.dirname(del_path), exist_ok=True)
        try:
            shutil.move(manga_path, del_path)
        except Exception as e:
            print(f"Error moving {manga_path} to {del_path}: {str(e)}")
        return
    
    # 조건 2: 압축파일이 있고 이미지가 2개 이하인 경우
    if zip_files and len(images) <= 2:
        for img in images:
            os.remove(img)
        
        for zip_file in zip_files:
            try:
                extract_first_image(zip_file, manga_path)
            except Exception as e:
                print(f"Error processing {zip_file}: {str(e)}")
    
    # 조건 3: 이미지 파일 이름 변경
    rename_images(manga_path)

def extract_first_image(zip_file, dest_dir):
    """압축 파일에서 첫 번째 이미지 추출"""
    ext = os.path.splitext(zip_file)[1].lower()
    
    try:
        if ext == '.zip':
            with zipfile.ZipFile(zip_file, 'r') as zf:
                for file in zf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        zf.extract(file, dest_dir)
                        break
        elif ext == '.7z':
            with py7zr.SevenZipFile(zip_file, 'r') as sz:
                for file in sz.getnames():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        sz.extract(dest_dir, [file])
                        break
        elif ext == '.rar':
            with rarfile.RarFile(zip_file, 'r') as rf:
                for file in rf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        rf.extract(file, dest_dir)
                        break
        else:
            patoolib.extract_archive(zip_file, outdir=dest_dir, interactive=False)
    except Exception as e:
        print(f"Error extracting {zip_file}: {str(e)}")

def rename_images(directory):
    """이미지 파일 이름을 숫자 형식으로 변경"""
    images = [f for f in os.listdir(directory) if any(f.lower().endswith(ext) for ext in img_extensions)]
    images.sort()
    
    for i, img in enumerate(images, 1):
        ext = os.path.splitext(img)[1].lower()
        if ext == '.webp':
            new_name = f"{i}.webp"
        else:
            new_name = f"{i}.jpg"
        old_path = os.path.join(directory, img)
        new_path = os.path.join(directory, new_name)
        os.rename(old_path, new_path)

def update_database(manga_path):
    """SQLite 데이터베이스 업데이트"""
    conn = sqlite3.connect('manga.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS manga_dirs (
        folder TEXT PRIMARY KEY,
        images TEXT,
        zip_files TEXT,
        file_date TEXT,
        create_date TEXT,
        update_date TEXT,
        tag TEXT
    )
    ''')
    
    images, zip_files = get_file_list(manga_path)
    relative_path = os.path.relpath(manga_path, '/home/manga')
    
    if images:
        file_date = min(datetime.fromtimestamp(os.path.getmtime(img), KST) 
                      for img in images).strftime('%Y-%m-%d %H:%M:%S')
    else:
        file_date = ''
        
    now = datetime.now(KST).strftime('%Y-%m-%d %H:%M:%S')
    
    cursor.execute('''
    INSERT OR REPLACE INTO manga_dirs 
    (folder, images, zip_files, file_date, create_date, update_date, tag)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        relative_path,
        ','.join([os.path.basename(img) for img in images]),
        ','.join([os.path.basename(zip_f) for zip_f in zip_files]),
        file_date,
        now,
        now,
        ''
    ))
    
    conn.commit()
    conn.close()

def main():
    manga_root = '/home/manga'
    
    # 모든 장르 디렉토리 순회
    for genre_dir in os.listdir(manga_root):
        genre_path = os.path.join(manga_root, genre_dir)
        if not os.path.isdir(genre_path):
            continue
            
        # 각 장르 디렉토리 내의 만화 디렉토리 순회
        for manga_dir in os.listdir(genre_path):
            manga_path = os.path.join(genre_path, manga_dir)
            if not os.path.isdir(manga_path):
                continue
                
            # 만화 디렉토리 처리
            process_manga_directory(manga_path)
            
            # 데이터베이스 업데이트
            if os.path.exists(manga_path):  # 삭제되지 않은 경우만
                update_database(manga_path)

if __name__ == "__main__":
    main()

In [13]:
import redis
import json
import os, sys
import time  # time.sleep을 사용하기 위해 추가
from datetime import datetime, timedelta


REDIS_CLIENT = redis.Redis(
        host='localhost',
        port=6379,
        db = 0,
        decode_responses=True
    )

def read_json_lines(file_path):
    data_list = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():  # 빈 줄 제외
                    try:
                        data = json.loads(line)
                        data_list.append(data)
                        # # NWS가 아닌 데이터만 추가
                        # if data['header']['tr_cd'] != 'NWS':
                        #     data_list.append(data)
                    except json.JSONDecodeError as e:
                        print(f"JSON 파싱 오류: {e}")
                        continue
    except Exception as e:
        print(f"파일 읽기 오류: {e}")
    
    return data_list

def redis_publish(channel, data):
    REDIS_CLIENT.publish(channel, json.dumps(data, ensure_ascii=False))
    print(json.dumps(data, ensure_ascii=False))

def test_redis_juga_publish():
    file_path = '/home/kds/k202410/Stock/websocket_response.json'
    data = read_json_lines(file_path)

    index = 0
    data_length = len(data)
    # 1분 후의 종료 시간 설정
    end_time = datetime.now() + timedelta(minutes=1)

    while datetime.now() < end_time:  # 현재 시간이 종료 시간보다 작을 때만 실행
        if data_length > 0 and data[index]['header']['tr_cd'] != 'NWS':  # 데이터가 있는 경우에만 실행
            channel = f'stock:{data[index]["header"]["tr_key"]}'
            print(data[index])
            redis_publish(channel, data[index])
            index = (index + 1) % data_length
            time.sleep(0.5)

    print("1분이 경과하여 프로그램을 종료합니다.")

In [16]:
test_redis_juga_publish()

{"header": {"tr_cd": "S3_", "tr_key": "042700"}, "body": {"mdchecnt": "6827", "sign": "2", "mschecnt": "7202", "mdvolume": "286858", "w_avrg": "84096", "cpower": "96.90", "offerho": "83700", "cvolume": "1", "high": "85900", "bidho": "83600", "low": "81900", "price": "83700", "cgubun": "+", "value": "47863", "change": "1200", "shcode": "042700", "chetime": "141846", "opentime": "090003", "lowtime": "090020", "volume": "569139", "drate": "1.45", "hightime": "100506", "jnilvolume": "373568", "msvolume": "277959", "open": "82000", "status": "00"}}
{"header": {"tr_cd": "K3_", "tr_key": "033100"}, "body": {"mdchecnt": "3311", "sign": "5", "mschecnt": "2584", "mdvolume": "172777", "w_avrg": "45736", "cpower": "42.06", "offerho": "45850", "cvolume": "1", "high": "46950", "bidho": "45800", "low": "44800", "price": "45850", "cgubun": "+", "value": "11311", "change": "1500", "shcode": "033100", "chetime": "142031", "opentime": "090013", "lowtime": "112838", "volume": "247312", "drate": "-3.17", "

In [33]:
t = b'{"t3102OutBlock1":[{"sBody":"t3102OutBlock1  <TABLE cellSpacing=\\"5\\" cellPadding=\\"0\\" width=\\"670\\" align=\\"CENTER\\" bgColor=\\"ffffff\\" border=\\"0\\"><TR><TD style=\\"PADDING-RIGHT: 2px; PADDING-LEFT: 2px; PADDING-BOTTOM: 2px; PADDING-TOP: 2px;\\" bgColor=\\"ffffff\\"><TABLE cellSpacing=\\"5\\" cellPadding=\\"0\\" bgColor=\\"ffffff\\" border=\\"0\\"><TR><TD><IMGsrc=\\"http://img.edaily.co.kr/images/photo/files/NP/S/2024/12/PS24122700706.jpg\\" border=\\"0\\"></TD></TR></TABLE></TD></TR></TABLE>\\r\\n[\xec\x9d\xb4\xeb\x8d\xb0\xec\x9d\xbc\xeb\xa6\xac \xea\xb9\x80\xed\x83\x9c\xed\x98\x95 \xea\xb8\xb0\xec\x9e\x90] \xec\xa0\x95\xec\xb2\xad\xeb\x9e\x98 \xea\xb5\xad\xed\x9a\x8c \xed\x83\x84\xed\x95\xb5\xec\x86\x8c\xec\xb6\x94\xeb\x8b\xa8\xec\x9e\xa5\xec\x9d\xb4 27\xec\x9d\xbc \xec\x98\xa4\xed\x9b\x84 \xec\x84\x9c\xec\x9a\xb8 \xed\x97\x8c\xeb\xb2\x95\xef\xbf\xbd\xef\xa6\xaf\xed\x93\xac\xef\xbf\xbd \xec\x86\x8c\xec\x8b\xac\xed\x8c\x90\xec\xa0\x95\xec\x97\x90\xec\x84\x9c \xec\x97\xb4\xeb\xa6\xb0 \xec\x9c\xa4\xec\x84\x9d\xec\x97\xb4 \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9 \xed\x83\x84\xed\x95\xb5 \xec\x82\xac\xea\xb1\xb4\xec\x97\x90 \xeb\x8c\x80\xed\x95\x9c \xec\xb2\xab \xeb\xb3\x80\xeb\xa1\xa0\xec\xa4\x80\xeb\xb9\x84\xea\xb8\xb0\xec\x9d\xbc \xec\xb6\x9c\xec\x84\x9d\xec\x97\x90 \xec\x95\x9e\xec\x84\x9c \xec\x9e\x85\xec\x9e\xa5\xec\x9d\x84 \xeb\xb0\x9c\xed\x91\x9c\xed\x95\x98\xea\xb3\xa0 \xef\xbf\xbd\xe7\x8d\xb5\xef\xbf\xbd. \\r\\n\\r\\n\xec\x99\xbc\xec\xaa\xbd\xeb\xb6\x80\xed\x84\xb0 \xec\xb5\x9c\xea\xb8\xb0\xec\x83\x81 \xeb\x8d\x94\xeb\xb6\x88\xec\x96\xb4\xeb\xaf\xbc\xec\xa3\xbc\xeb\x8b\xb9 \xec\x9d\x98\xec\x9b\x90, \xec\xa0\x95\xec\xb2\xad\xeb\x9e\x98 \xeb\x8b\xa8\xec\x9e\xa5, \xeb\xb2\x95\xeb\xa5\xa0\xeb\x8c\x80\xeb\xa6\xac\xec\x9d\xb8\xeb\x8b\xa8 \xea\xb3\xb5\xeb\x8f\x99 \xeb\x8c\x80\xed\x91\x9c \xea\xb9\x80\xec\x9d\xb4\xec\x88\x98 \xec\xa0\x84 \xed\x97\x8c\xeb\xb2\x95\xec\x9e\xac\xed\x8c\x90\xea\xb4\x80.\\r\\n<p style=\'clear:both;\'>\\r\\n\xef\xbc\x9c\xe2\x93\x92\xec\xa2\x85\xed\x95\xa9 \xea\xb2\xbd\xec\xa0\x9c\xec\xa0\x95\xeb\xb3\xb4 \xeb\xaf\xb8\xeb\x94\x94\xec\x96\xb4 \xec\x9d\xb4\xeb\x8d\xb0\xec\x9d\xbc\xeb\xa6\xac - \xeb\xac\xb4\xeb\x8b\xa8\xec\xa0\x84\xec\x9e\xac & \xec\x9e\xac\xeb\xb0\xb0\xed\x8f\xac \xea\xb8\x88\xec\xa7\x80\xef\xbc\x9e</p>\\r\\n"}],"t3102OutBlock2":{"sTitle":"t3102OutBlock2  [\xed\x8f\xac\xed\x86\xa0] \xec\xa0\x95\xec\xb2\xad\xeb\x9e\x98 \xea\xb5\xad\xed\x9a\x8c \xed\x83\x84\xed\x95\xb5\xec\x86\x8c\xec\xb6\x94\xeb\x8b\xa8\xec\x9e\xa5 \xea\xb8\xb0\xec\x9e\x90\xed\x9a\x8c\xea\xb2\xac _dx\xeb\x8f\x9b\xef\xbf\xbd    \xef\xbf\xbd0  1     `\xe2\x94\xba                                                                                                                                                                                                   1    \xef\xbf\xbd\xef\xbf\xbd\\u0010\xef\xbf\xbd \\u0006      \xef\xbf\xbd\xef\xbf\xbd\\u0011\xef\xbf\xbd \\u0013\\u001D\xef\xbf\xbd"},"rsp_cd":"00000","rsp_msg":"\xec\xa1\xb0\xed\x9a\x8c\xec\x99\x84\xeb\xa3\x8c"}'
import re

# sBody 내용 추출 정규식
pattern = r'"sBody":"(.*?)(?="})'

# 모든 매치 찾기
matches = re.findall(pattern, t.decode('utf-8'))

# 결과 출력
for match in matches:
    print("=== sBody ===")
    print(match)
    print()

=== sBody ===
t3102OutBlock1  <TABLE cellSpacing=\"5\" cellPadding=\"0\" width=\"670\" align=\"CENTER\" bgColor=\"ffffff\" border=\"0\"><TR><TD style=\"PADDING-RIGHT: 2px; PADDING-LEFT: 2px; PADDING-BOTTOM: 2px; PADDING-TOP: 2px;\" bgColor=\"ffffff\"><TABLE cellSpacing=\"5\" cellPadding=\"0\" bgColor=\"ffffff\" border=\"0\"><TR><TD><IMGsrc=\"http://img.edaily.co.kr/images/photo/files/NP/S/2024/12/PS24122700706.jpg\" border=\"0\"></TD></TR></TABLE></TD></TR></TABLE>\r\n[이데일리 김태형 기자] 정청래 국회 탄핵소추단장이 27일 오후 서울 헌법�羚퓬� 소심판정에서 열린 윤석열 대통령 탄핵 사건에 대한 첫 변론준비기일 출석에 앞서 입장을 발표하고 �獵�. \r\n\r\n왼쪽부터 최기상 더불어민주당 의원, 정청래 단장, 법률대리인단 공동 대표 김이수 전 헌법재판관.\r\n<p style='clear:both;'>\r\n＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞</p>\r\n



### video 폴더
/home/video/


In [ ]:
pip install pytz py7zr rarfile patool

APP_KEY: PSU9hy7su4wgvmcjdMHTyLtH5r36qMYQeku8
APP_SECRET: nSMC08OT2GSHn18GKX5IsoDWuwgytUfh

In [ ]:
': '(주)유한양행', '본사주소': '서울시 동작구 노량진로 74', '본사전화번호': '02-828-0181', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 1000, '주식수': 80209064, 'Homepage': 'www.yuhan.co.kr', '그룹명': '', '외국인': '21.33', '주담전화': '02-828-0181', '자본금': '802', '시가총액': '94887', '배당금': '450', '배당수익율': '0.38', '현재가': 118300, '전일종가': 119500, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A000100', '결산년월': '202312', '결산구분': 'D', 'PER': '102.97', 'EPS': '1149', 'PBR': '4.31', 'ROA': '4.00', 'ROE': '4.77', 'EBITDA': '88877793080.0', 'EVEBITDA': '60.49', '액면가': '1000.00', 'SPS': '23247.13', 'CPS': '1608.60', 'BPS': '27437', 'T.PER': '85.37', 'T.EPS': '1386', 'PEG': '0.00', 'T.PEG': '557.42', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 반도체 및 관련장비', '시장구분': '1', '시장구분명': '거래소', '한글 기업명': '에스케이하이닉스(주)', '본사주소': '경기도 이천시 부발읍 경충대로 2091', '본사전화번호': '031-630-4114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 728002365, 'Homepage': 'www.skhynix.com', '그룹명': '에스케이', '외국인': '54.71', '주담전화': '031-5185-4114', '자본금': '36577', '시가총액': '1246340', '배당금': '1200', '배당수익율': '0.69', '현재가': 171200, '전일종가': 173900, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A000660', '결산년월': '202312', '결산구분': 'D', 'PER': '-25.77', 'EPS': '-6643', 'PBR': '2.19', 'ROA': '-5.24', 'ROE': '-8.43', 'EBITDA': '41032700000.00', 'EVEBITDA': '31.18', '액면가': '5000.00', 'SPS': '37966.91', 'CPS': '5411.00', 'BPS': '78009', 'T.PER': '12.15', 'T.EPS': '14095', 'PEG': '0.00', 'T.PEG': '0.00', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 자동차', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '현대자동차(주)', '본사주소': '서울시 서초구 헌릉로 12 (양재동)', '본사전화번호': '02-3464-1114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 209416191, 'Homepage': 'www.hyundai.com', '그룹명': '현대자동차', '외국인': '39.47', '주담전화': '02-3464-1114', '자본 금': '11580', '시가총액': '442915', '배당금': '11400', '배당수익율': '5.38', '현재가': 211500, '전일종가': 212000, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기 업코드': 'A005380', '결산년월': '202312', '결산구분': 'D', 'PER': '7.90', 'EPS': '26758', 'PBR': '0.72', 'ROA': '8.72', 'ROE': '12.60', 'EBITDA': '10040398000.00', 'EVEBITDA': '4.03', '액면가': '5000.00', 'SPS': '284358.80', 'CPS': '39036.62', 'BPS': '293026', 'T.PER': '6.27', 'T.EPS': '33716', 'PEG': '7.89', 'T.PEG': '24.12', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS  반도체 및 관련장비', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '삼성전자(주)', '본사주소': '경기도 수원시 영통구 삼성로 129 (매탄동)', '본사전화번호': '031-200-1114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 100, '주식수': 5969782550, 'Homepage': 'www.samsung.com', '그룹명': '삼성', '외국인': '50.46', '주담전화': '031-200-1114', '자본금': '7780', '시가총액': '3187864', '배당금': '1444', '배당수익율': '2.71', '현 재가': 53400, '전일종가': 53200, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A005930', '결산년월': '202312', '결산구분': 'D', 'PER': '14.28', 'EPS': '3739', 'PBR': '1.42', 'ROA': '9.12', 'ROE': '11.70', 'EBITDA': '16707486000.00', 'EVEBITDA': '32.47', '액면가': '100.00', 'SPS': '25082.05', 'CPS': '7895.41', 'BPS': '37654', 'T.PER': '15.26', 'T.EPS': '3499', 'PEG': '0.00', 'T.PEG': '0.00', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 상업서비스', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '한화에어로스페이스(주)', '본사주소': '경남 창원시 성산구 창원대로 1204 (성주동)', '본사전화번호': '055-260-2114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 45581161, 'Homepage': 'www.hanwhaaerospace.co.kr', '그룹명': '한화', '외국인': '43.59', '주담전화': '02-729-5200', '자본금': '2404', '시가총액': '165688', '배당 금': '1800', '배당수익율': '0.55', '현재가': 363500, '전일종가': 326500, '위험고지구분1_정리매매': '0', '위험고지구 분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A012450', '결산년월': '202312', '결산구분': 'D', 'PER': '28.36', 'EPS': '12818', 'PBR': '5.85', 'ROA': '5.68', 'ROE': '22.38', 'EBITDA': '639908074660', 'EVEBITDA': '13.23', '액면가': '5000.00', 'SPS': '98340.76', 'CPS': '15203.82', 'BPS': '62147', 'T.PER': '27.86', 'T.EPS': '13048', 'PEG': '25.41', 'T.PEG': '1547.03', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': ' 정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 무선통신', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '에스케이텔레콤(주)', '본사주소': '서울시 중구 을지로 65 SK T타워', '본사전화번호': '02-6100-2114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 100, '주식수': 214790053, 'Homepage': 'www.sktelecom.com', '그룹명': '에스케이', '외국인': '42.04', '주담전화': '02-6100-2114', '자본금': '305', '시가총액': '120497', '배당금': '3540', '배당수익율': '6.41', '현재가': 56100, '전일종가': 55200, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A017670', '결산년월': '202312', '결산구분': 'D', 'PER': '11.58', 'EPS': '4843', 'PBR': '1.14', 'ROA': '4.12', 'ROE': '10.18', 'EBITDA': '42891970000.00', 'EVEBITDA': '4.49', '액면가': '100.00', 'SPS': '57528.85', 'CPS': '17790.16', 'BPS': '49070', 'T.PER': '10.93', 'T.EPS': '5130', 'PEG': '52.94', 'T.PEG': '184.02', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 상업은행', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '중소기업은행', '본사주소': '서울시 중구 을지로 79 (을지로 2가 50), 중 소기업은행본점건물', '본사전화번호': '02-729-6114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', ' 주당액면가': 5000, '주식수': 797425869, 'Homepage': 'www.ibk.co.kr', '그룹명': '', '외국인': '14.40', '주담전화': '', '자본금': '39871', '시가총액': '114670', '배당금': '984', '배당수익율': '6.87', '현재가': 14380, '전일종가': 14330, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A024110', '결산년월': '202312', '결산구분': 'D', 'PER': '4.76', 'EPS': '3024', 'PBR': '0.39', 'ROA': '0.59', 'ROE': '8.50', 'EBITDA': '0.00', 'EVEBITDA': '0.00', '액면가': '5000.00', 'SPS': '21101.87', 'CPS': '3342.09', 'BPS': '36975', 'T.PER': '4.56', 'T.EPS': '3157', 'PEG': '0.00', 'T.PEG': '103.95', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 무선통신', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '(주)케이티', '본사주소': '경기도 성남시 분당구 불정로 90 (정자 동)', '본사전화번호': '070-4193-4036', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 252021685, 'Homepage': 'www.kt.com', '그룹명': '케이티', '외국인': '49.00', '주담전화': '070-4193-4036', '자본금': '15645', '시가총액': '110764', '배당금': '1960', '배당수익율': '4.47', '현재가': 43950, '전일종가': 43850, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A030200', '결산년월': '202312', '결산구분': 'D', 'PER': '12.24', 'EPS': '3592', 'PBR': '0.73', 'ROA': '3.07', 'ROE': '6.24', 'EBITDA': '44183720000.00', 'EVEBITDA': '3.52', '액면가': '5000.00', 'SPS': '70705.82', 'CPS': '16034.83', 'BPS': '59884', 'T.PER': '10.44', 'T.EPS': '4209', 'PEG': '53.64', 'T.PEG': '60.78', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 전기장비', '시장구분': '2', '시장구분명': '코스닥', '한글기업명': '제룡전기(주)', '본사주소': '서울시 광진구 아차 산로 628', '본사전화번호': '02-2204-6300', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 500, '주식수': 16062409, 'Homepage': 'www.cheryongelec.com', '그룹명': '', '외국인': '19.24', '주담전화': '02-2204-6310', '자본금': '80', '시가총액': '7622', '배당금': '500', '배당수익율': '1.09', '현재가': 47450, '전일종가': 45950, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A033100', '결산년월': '202312', '결산구분': 'D', 'PER': '13.52', 'EPS': '3510', 'PBR': '6.11', 'ROA': '45.74', 'ROE': '57.64', 'EBITDA': '71525127120.0', 'EVEBITDA': '3.81', '액면가': '500.00', 'SPS': '11451.65', 'CPS': '3595.79', 'BPS': '7770', 'T.PER': '8.63', 'T.EPS': '5501', 'PEG': '3.84', 'T.PEG': '15.21', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 담 배', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '(주)케이티앤지', '본사주소': '대전시 대덕구 벚꽃길 71', '본사전화번호': '042-939-5000', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 125362497, 'Homepage': 'www.ktng.com', '그룹명': '케이티앤지', '외국인': '44.49', '주담전화': '042-939-5000', '자본금': '9550', '시가총액': '134263', '배당금': '5200', '배당수익율': '4.86', '현재가': 107100, '전일종가': 107100, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A033780', '결산년월': '202312', '결산구분': 'D', 'PER': '18.16', 'EPS': '5898', 'PBR': '1.58', 'ROA': '7.84', 'ROE': '10.13', 'EBITDA': '10927250000.00', 'EVEBITDA': '10.04', '액면가': '5000.00', 'SPS': '26284.65', 'CPS': '7078.02', 'BPS': '67772', 'T.PER': '17.34', 'T.EPS': '6177', 'PEG': '0.00', 'T.PEG': '367.08', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 반도체 및 관련장비', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '한미반도체(주)', '본사주소': '인천시 서구 가좌로30번길 14(가좌동)', '본사전화번호': '032-571-9100', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 100, '주식수': 96614259, 'Homepage': 'www.hanmisemi.com', '그룹명': '', '외국인': '11.55', '주담전화': '032-571-9100', '자본금': '127', '시가총액': '84151', '배당금': '420', '배당수익율': '0.51', '현재가': 87100, '전일종가': 82500, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A042700', '결산년월': '202312', '결산구분': 'D', 'PER': '31.26', 'EPS': '2786', 'PBR': '14.33', 'ROA': '45.78', 'ROE': '56.10', 'EBITDA': '46165267330.0', 'EVEBITDA': '126.26', '액면가': '100.00', 'SPS': '1613.75', 'CPS': '2872.00', 'BPS': '6079', 'T.PER': '46.59', 'T.EPS': '1869', 'PEG': '15.56', 'T.PEG': '0.00', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 상업서비스', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '한국항공우주산업(주)', '본 사주소': '경남 사천시 사남면 공단1로 78', '본사전화번호': '055-851-1000', '최근결산년도': '2023', '결산월': '12', ' 결산년월': '202312', '주당액면가': 5000, '주식수': 97475107, 'Homepage': 'www.koreaaero.com', '그룹명': '', '외국인': '34.36', '주담전화': '055-851-1000', '자본금': '4874', '시가총액': '54391', '배당금': '500', '배당수익율': '0.91', '현재가': 55800, '전일종가': 54900, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A047810', '결산년월': '202312', '결산구분': 'D', 'PER': '23.50', 'EPS': '2375', 'PBR': '3.34', 'ROA': '3.13', 'ROE': '15.02', 'EBITDA': '369317152390', 'EVEBITDA': '12.79', '액면가': '5000.00', 'SPS': '38967.49', 'CPS': '3561.89', 'BPS': '16726', 'T.PER': '18.08', 'T.EPS': '3086', 'PEG': '27.78', 'T.PEG': '60.40', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 바이오', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '(주)셀트리온', '본사주소': '인천시 연수구 아카데미로 23', '본사전화번호': '032-850-5000', '최근결산년도': '2023', '결산월': '12', '결 산년월': '202312', '주당액면가': 1000, '주식수': 217078554, 'Homepage': 'www.celltrion.com', '그룹명': '셀트리온', '외국인': '22.13', '주담전화': '032-850-5000', '자본금': '2205', '시가총액': '391393', '배당금': '500', '배당수익율': '0.27', '현재가': 180300, '전일종가': 187500, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A068270', '결산년월': '202312', '결산구분': 'D', 'PER': '47.11', 'EPS': '3827', 'PBR': '2.06', 'ROA': '4.53', 'ROE': '5.31', 'EBITDA': '833736878610', 'EVEBITDA': '36.35', '액면가': '1000.00', 'SPS': '12728.31', 'CPS': '5153.45', 'BPS': '87361', 'T.PER': '55.18', 'T.EPS': '3268', 'PEG': '498.23', 'T.PEG': '0.00', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 금속 및 광물', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '(주)풍산', '본사주소': '경기도 평택시 포승읍 평택항로156번길 134', '본사전화번호': '02-3406-5114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 28024278, 'Homepage': 'www.poongsan.co.kr', '그룹명': '풍산', '외국인': '21.14', '주담전화': '02-3406-5114', '자본금': '1401', '시가총액': '14489', '배당금': '1200', '배당수익율': '2.40', '현재가': 51700, '전일종가': 49950, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A103140', '결산년월': '202312', '결산구분': 'D', 'PER': '9.27', 'EPS': '5577', 'PBR': '0.73', 'ROA': '4.90', 'ROE': '8.21', 'EBITDA': '256586661260', 'EVEBITDA': '5.22', '액면가': '5000.00', 'SPS': '110638.17', 'CPS': '7942.46', 'BPS': '70959', 'T.PER': '6.23', 'T.EPS': '8292', 'PEG': '0.00', 'T.PEG': '12.81', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 상업은행', '시장구분': '1', '시장구분명': '거래소', '한글기 업명': '(주)KB금융지주', '본사주소': '서울시 영등포구 국제금융로8길 26 (여의도동, 국민은행 여의도본점)', '본사전화번호': '02-2073-7114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액면가': 5000, '주식수': 393528423, 'Homepage': 'www.kbfg.com', '그룹명': '', '외국인': '76.83', '주담전화': '02-2073-7114', '자본금': '20906', '시가총액': '328203', '배당금': '3060', '배당수익율': '3.69', '현재가': 83400, '전일종가': 82900, '위험고지구분1_정 리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A105560', '결산년월': '202312', '결산구분': 'D', 'PER': '15.92', 'EPS': '5239', 'PBR': '1.28', 'ROA': '7.07', 'ROE': '8.61', 'EBITDA': '0.00', 'EVEBITDA': '0.00', '액면가': '5000.00', 'SPS': '86.76', 'CPS': '5253.05', 'BPS': '64950', 'T.PER': '15.29', 'T.EPS': '5455', 'PEG': '57.00', 'T.PEG': '370.56', '최근분기년도': '202409'}, 'rsp_cd': '00000', 'rsp_msg': '정상적으로 조회가 완료되었습니다.'}, {'t3320OutBlock': {'업종구분명': 'FICS 바이오', '시장구분': '1', '시장구분명': '거래소', '한글기업명': '삼성바이오로직스(주)', '본사주소': '인천시 연수구 송도바이오대로 300(송도동, 삼성바 이오로직스)', '본사전화번호': '032-455-3114', '최근결산년도': '2023', '결산월': '12', '결산년월': '202312', '주당액 면가': 2500, '주식수': 71174000, 'Homepage': 'www.samsungbiologics.com', '그룹명': '삼성', '외국인': '13.36', '주담 전화': '', '자본금': '1779', '시가총액': '664765', '배당금': '0', '배당수익율': '0.00', '현재가': 934000, '전일종가': 949000, '위험고지구분1_정리매매': '0', '위험고지구분2_투자위험': '0', '위험고지구분3_단기과열': '0'}, 
't3320OutBlock1': {'기업코드': 'A207940', '결산년월': '202312', '결산구분': 'D', 'PER': '70.28', 'EPS': '13290', 'PBR': '8.07', 'ROA': '7.88', 'ROE': '12.18', 'EBITDA': '14590843848.40', 'EVEBITDA': '36.62', '액면가': '2500.00', 'SPS': '41289.74', 'CPS': '16871.56', 'BPS': '115753', 'T.PER': '62.32', 'T.EPS': '14988', 'PEG': '197.63', 'T.PEG': '487.91', '최 근분기년도': '202409'}, 'r

In [9]:
from math import ceil
n = '645681312235842845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845'
print(len(n))
k = 0
for i in range(ceil(len(n)/300)):
    start = i*300
    end = (i+1)*300
    print(start, end)
    print(n[start:end])
    print(len(n[start:end]))
    k += len(n[start:end])
    print('-'*100)
print(k)

1470
0 300
645681312235842845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681
300
----------------------------------------------------------------------------------------------------
300 600
312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681312845645681
300
----------------------------------------------------------------------------------------------------
600 900
31284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284564568131284

In [ ]:
{'날짜': '20250106', 
 '시가': 95900, 
 '고가': 99400, 
 '저가': 95800, 
 '종가': 97100, 
 '거래량': 563340, 
 '거래대금': 55042, 
 '수정구분': 0, 
 '수정비율': '0.00', 
 '수정주가반영항목': 0, 
 '수정비율반영거래대금': 0, 
 '종가등락구분(1:상한2:상승3:보합4:하한5:하락주식일만사용)': '2'}

In [ ]:
multi_price= [
    {'종목코드': '033100', 
     '종목명': '제룡전기', 
     '현재가': 48500, 
     '전일대비구분': '2', 
     '전일대비': 450, 
     '등락율': '0.94', 
     '누적거래량': 230872, 
     '매도호가': 48500, '매수호가': 48450, 
     '체결수량': 1, '체결강도': '72.29', 
     '시가': 48600, 
     '고가': 49300, 
     '저가': 47750, 
     '거래대금(백만)': 11168, '우선매도잔량': 623, '우선매수잔량': 34, 
     '총매도잔량': 10922, '총매수잔량': 18418, '전일종가': 48050, 
     '상한가': 62400, '하한가': 33650}]